In [1]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

In [2]:
import sys
import os

sys.path.insert(0, os.path.abspath(".."))
sys.path.append(os.path.abspath("../backend"))
sys.path.append(os.path.abspath(""))

In [3]:
from rdflib.plugins.stores.sparqlstore import SPARQLStore
from backend.ontology import Graph, OntologyManager, OntologyConfig
from backend.datasetmatcher import DatasetManager
from backend.explorative.explorative_support import GuidanceManager
from backend.explorative.llm_query import LLMQuery
from backend.explorative.topic_init import TopicInitator

base_path = "../data"
onto_path = f"{base_path}/hero-ontology/hereditary_clinical.ttl"


# graph = Graph().parse(onto_path, format="turtle")
# graph.bind("bto", "http://www.semanticweb.org/ontologies/2020/3/bto#")

# store = SPARQLStore(
#     "http://localhost:7200/repositories/dpedia",
#     method="POST_FORM",
#     params={"infer": False, "sameAs": False},
# )

# store = SPARQLStore(
#     "http://localhost:3030/dbpedia/query",
#     method="POST_FORM",
#     params={"infer": False, "sameAs": False},
# )
store = SPARQLStore(
    "http://localhost:7012/",
    method="POST_FORM",
    params={"infer": False, "sameAs": False},
    retries=10,
)
graph = Graph(store=store)

config = OntologyConfig()

ontology_manager = OntologyManager(config, graph)
dataset_manager = DatasetManager(ontology_manager)
dataset_manager.initialise(glob_path="data/datasets/ALS/**/*.csv")

# ontology_manager.load_full_graph()


-- Loading and merging datasets
0 datasets loaded


In [4]:
guidance_man = GuidanceManager(ontology_manager)
guidance_man.llama_model
print(guidance_man.identifier)
llm_query = LLMQuery(guidance_man)
topic_init = TopicInitator(guidance_man)

Loading LLM model NousResearch/Hermes-3-Llama-3.1-8B-GGUF None


llama_load_model_from_file: using device Metal (Apple M2 Pro) - 21845 MiB free
llama_model_loader: loaded meta data with 27 key-value pairs and 292 tensors from /Users/benedikt/.cache/huggingface/hub/models--NousResearch--Hermes-3-Llama-3.1-8B-GGUF/snapshots/307a5dfb59aa38d88b6cfd32f44b8ad7c1da9fb8/./Hermes-3-Llama-3.1-8B.Q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Hermes 3 Llama 3.1 8B
llama_model_loader: - kv   3:                       general.organization str              = NousResearch
llama_model_loader: - kv   4:                           general.basename str              = Hermes-3-Llama-3.1
llama_model_loa

bb3ed8536753defab35ef3a106ad6561d70669359971c082b1b743c485365110


Metal : EMBED_LIBRARY = 1 | CPU : NEON = 1 | ARM_FMA = 1 | FP16_VA = 1 | MATMUL_INT8 = 1 | ACCELERATE = 1 | AARCH64_REPACK = 1 | 
Model metadata: {'tokenizer.chat_template': "{% if not add_generation_prompt is defined %}{% set add_generation_prompt = false %}{% endif %}{% for message in messages %}{{'<|im_start|>' + message['role'] + '\n' + message['content'] + '<|im_end|>' + '\n'}}{% endfor %}{% if add_generation_prompt %}{{ '<|im_start|>assistant\n' }}{% endif %}", 'tokenizer.ggml.padding_token_id': '128040', 'tokenizer.ggml.eos_token_id': '128040', 'general.type': 'model', 'tokenizer.ggml.bos_token_id': '128000', 'tokenizer.ggml.pre': 'llama-bpe', 'tokenizer.ggml.model': 'gpt2', 'llama.embedding_length': '4096', 'llama.vocab_size': '128256', 'llama.attention.head_count_kv': '8', 'general.file_type': '7', 'llama.block_count': '32', 'general.size_label': '8B', 'llama.feed_forward_length': '14336', 'llama.attention.layer_norm_rms_epsilon': '0.000010', 'general.architecture': 'llama', '

In [5]:
# topic_init.initate()

In [6]:
llm_query.initiate(n_queries=10, force=True)

  0%|          | 0/10 [00:00<?, ?it/s]llama_perf_context_print:        load time =    7480.76 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   212 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    11 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =    8050.61 ms /   223 tokens
 10%|█         | 1/10 [00:08<01:17,  8.61s/it]Llama.generate: 167 prefix-match hit, remaining 72 prompt tokens to eval
llama_perf_context_print:        load time =    7480.76 ms
llama_perf_context_print: prompt eval time =       0.00 ms /    72 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    20 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =    1343.34 ms /    92 tokens
 20%|██        | 2/10 [00:10<00:37,  4.66s/it]Llama.generate: 156 prefix-

[('The birth place of an Organisation member is a stream.',
  EnrichedEntitiesRelations(relations=[EnrichedRelation(entity='Organisation member', relation='birth place', target='stream', link=SubjectLink(link_id=54006, label='birth place', from_id='<http://dbpedia.org/ontology/OrganisationMember>', link_type='owl:ObjectProperty', to_id='<http://dbpedia.org/ontology/Stream>', to_proptype=None, property_id='<http://dbpedia.org/ontology/birthPlace>', from_subject=Subject(subject_id='<http://dbpedia.org/ontology/OrganisationMember>', label='Organisation member', spos={'rdf:type': Property(property='rdf:type', label=None, values=[PropertyValue(value='owl:Class', label=None)]), 'rdfs:label': Property(property='rdfs:label', label=None, values=[PropertyValue(value='Organisation member', label=None)]), 'rdfs:subClassOf': Property(property='rdfs:subClassOf', label=None, values=[PropertyValue(value='<http://dbpedia.org/ontology/Person>', label='person')])}, subject_type='class', refcount=0, desce